In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java, isJVMStarted

C:\Users\mesut\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv("yorumsepeti.csv", sep = ";")
df = df.dropna()
df = df[["review"]]
df = df.values.tolist()

In [3]:
a = -1
for i in df:
    a +=1
    for j in i:
        if len(j) <=4:
            del df[a]
liste = []
for i in df:
    for j in i:
        j = j.lower()
        words = nltk.word_tokenize(j)
        new_words= [word for word in words if word.isalnum()]
        liste.append(new_words)

In [4]:
ZEMBEREK_PATH = r'C:\Users\mesut\zemberek-full.jar'
def init_jvm(jvmpath=ZEMBEREK_PATH):
    if isJVMStarted():
        print("gbgbgbgbg")
    else:
        print("nftntf")
        startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (jvmpath))
init_jvm()

nftntf


In [5]:
TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
morphology = TurkishMorphology.createWithDefaults()

In [6]:
b = " "
c = -1
for i in liste:
    c+=1
    a = ""
    for j in i:
        a += j + b
    liste[c] = a
liste2 = []
liste = list(filter(None, liste))

In [7]:
# lemmatization
for i in liste:
    analysis: java.util.ArrayList = (
        morphology.analyzeAndDisambiguate(i).bestAnalysis()
        )
    pos: List[str] = []
    for i, analysis in enumerate(analysis, start=1):
        f'\nAnalysis {i}: {analysis}',
        f'\nPrimary POS {i}: {analysis.getPos()}'
        f'\nPrimary POS (Short Form) {i}: {analysis.getPos().shortForm}'
       
        pos.append(
            f'{str(analysis.getLemmas()[0])}'
            )
    liste2.append(pos)

In [8]:
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 16
W2V_MIN_COUNT = 10
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

In [9]:
w2v_model.build_vocab(liste2)
w2v_model.train(liste2, total_examples=len(liste2), epochs=W2V_EPOCH)

(6849737, 10281344)

In [10]:
w2v_model.wv.most_similar("tatlı")

[('kurabiye', 0.6363350749015808),
 ('puding', 0.5664054751396179),
 ('kahve', 0.4959833025932312),
 ('dondurma', 0.49025481939315796),
 ('düşünce', 0.4830177128314972),
 ('tulumba', 0.46871015429496765),
 ('donut', 0.4545120298862457),
 ('sürpriz', 0.4540376365184784),
 ('jest', 0.45127564668655396),
 ('sütlaç', 0.4505675137042999)]

In [11]:
w2v_model.wv.most_similar("zaman")

[('süre', 0.4767528474330902),
 ('daim', 0.467551589012146),
 ('se', 0.4463024437427521),
 ('şey', 0.44054511189460754),
 ('herzaman', 0.437669575214386),
 ('haftasonu', 0.42593297362327576),
 ('sure', 0.4239300787448883),
 ('yön', 0.4140171706676483),
 ('hafta', 0.3945591449737549),
 ('yıl', 0.39436179399490356)]

In [12]:
w2v_model.wv.most_similar("yemek")

[('yiyecek', 0.4804593622684479),
 ('çorba', 0.38541296124458313),
 ('sodexo', 0.34751272201538086),
 ('konum', 0.3406924307346344),
 ('ürün', 0.33610275387763977),
 ('pilav', 0.33289384841918945),
 ('yardım', 0.3179759085178375),
 ('çay', 0.31706684827804565),
 ('restorant', 0.31647875905036926),
 ('cihaz', 0.3017111122608185)]

In [ ]:
## MESUT DEMİREL